In [3]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import files
uploaded = files.upload()

Saving S Parameter Plot 1.csv to S Parameter Plot 1.csv


In [5]:
data = pd.read_csv('S Parameter Plot 1.csv')
data.head()

,d,h,rad,s1,freq,"S(1,1)","S(2,1)"
0,0.15,2.0,2.5,2.9,0.000,-0.000098,-108.071415
1,0.15,2.0,2.5,2.9,0.045,-0.000220,-70.238113
2,0.15,2.0,2.5,2.9,0.090,-0.000338,-64.315271
3,0.15,2.0,2.5,2.9,0.135,-0.000437,-60.835543
4,0.15,2.0,2.5,2.9,0.180,-0.000524,-58.367403


In [6]:
data.describe()

,d,h,rad,s1,freq,"S(1,1)","S(2,1)"
count,36090.000000,36090.000000,36090.000000,36090.000000,36090.000000,36090.000000,36090.000000
mean,0.156111,2.003333,5.682222,3.043333,9.000000,-2.238380,-19.964195
std,0.057653,0.031447,1.939247,0.110606,5.209199,4.532916,17.206254
min,0.150000,2.000000,2.500000,2.900000,0.000000,-56.744734,-108.802741
25%,0.150000,2.000000,4.000000,2.900000,4.500000,-2.316034,-30.833675
50%,0.150000,2.000000,5.650000,3.000000,9.000000,-0.105933,-17.840675
75%,0.150000,2.000000,7.300000,3.100000,13.500000,-0.017828,-4.201232
max,0.700000,2.300000,9.100000,3.200000,18.000000,0.657448,-0.036995


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36090 entries, 0 to 36089
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   d       36090 non-null  float64
 1   h       36090 non-null  float64
 2   rad     36090 non-null  float64
 3   s1      36090 non-null  float64
 4   freq    36090 non-null  float64
 5   S(1,1)  36090 non-null  float64
 6   S(2,1)  36090 non-null  float64
dtypes: float64(7)
memory usage: 1.9 MB


In [8]:
x_train = data.iloc[:,5:]
x_train.head()

,"S(1,1)","S(2,1)"
0,-0.000098,-108.071415
1,-0.000220,-70.238113
2,-0.000338,-64.315271
3,-0.000437,-60.835543
4,-0.000524,-58.367403


In [9]:
y_train = data.iloc[:, :5]
y_train.head()

,d,h,rad,s1,freq
0,0.15,2.0,2.5,2.9,0.000
1,0.15,2.0,2.5,2.9,0.045
2,0.15,2.0,2.5,2.9,0.090
3,0.15,2.0,2.5,2.9,0.135
4,0.15,2.0,2.5,2.9,0.180


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

In [13]:
def build_generator(input_dim, output_dim):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(input_dim,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(output_dim, activation='linear'))
    return model

In [14]:
def build_discriminator(output_dim):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(output_dim,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [15]:
input_dim = 2
output_dim = 5

In [16]:
generator = build_generator(input_dim, output_dim)
discriminator = build_discriminator(output_dim)

In [17]:
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002))
discriminator.trainable = False  # Freeze discriminator during GAN training
gan_input = tf.keras.Input(shape=(input_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.0002))

In [49]:
num_samples = 30676
epochs = 10
batch_size = 128
train_output = y_train
train_input = x_train

In [50]:
train_output_numpy = train_output.to_numpy()
train_input_numpy = train_input.to_numpy()
num_samples, output_dim = train_output_numpy.shape
num_samples

36090

In [51]:
Loss = {}

In [52]:
for epoch in range(epochs):
    for _ in range(num_samples // batch_size):
        # Train discriminator on real data
        # real_data = train_output_numpy[np.random.randint(0, num_samples, batch_size)]
        # real_labels = np.ones((batch_size, 1))
        real_indices = np.random.randint(0, num_samples, batch_size)
        real_data = train_output_numpy[real_indices]

        # Generate fake data and train discriminator on fake data
        noise = np.random.rand(batch_size, input_dim)
        fake_data = generator.predict(noise)
        combined_data = np.concatenate([real_data, fake_data])
        combined_labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

        d_loss = discriminator.train_on_batch(combined_data, combined_labels)

        # Train the generator (via the GAN model)
        noise = np.random.rand(batch_size, input_dim)
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress
    print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")
    Loss[epoch] = [d_loss, g_loss]


4/4 [==============================] - 0s 3ms/step
Epoch 0, D Loss: 0.547461748123169, G Loss: 0.3411710560321808
4/4 [==============================] - 0s 5ms/step
Epoch 1, D Loss: 0.5741134285926819, G Loss: 0.34120094776153564
4/4 [==============================] - 0s 5ms/step
Epoch 2, D Loss: 0.736081600189209, G Loss: 0.3218727707862854
4/4 [==============================] - 0s 3ms/step
Epoch 3, D Loss: 0.6893937587738037, G Loss: 0.23225340247154236
4/4 [==============================] - 0s 4ms/step
Epoch 4, D Loss: 0.6210287809371948, G Loss: 0.31374070048332214
4/4 [==============================] - 0s 5ms/step
Epoch 5, D Loss: 0.9252649545669556, G Loss: 0.24690113961696625
4/4 [==============================] - 0s 7ms/step
Epoch 6, D Loss: 0.3989475965499878, G Loss: 0.4572628438472748
4/4 [==============================] - 0s 4ms/step
Epoch 7, D Loss: 0.8134912252426147, G Loss: 0.20236200094223022
4/4 [==============================] - 0s 3ms/step
Epoch 8, D Loss: 0.6844345

In [53]:
Loss

{0: [0.547461748123169, 0.3411710560321808],
 1: [0.5741134285926819, 0.34120094776153564],
 2: [0.736081600189209, 0.3218727707862854],
 3: [0.6893937587738037, 0.23225340247154236],
 4: [0.6210287809371948, 0.31374070048332214],
 5: [0.9252649545669556, 0.24690113961696625],
 6: [0.3989475965499878, 0.4572628438472748],
 7: [0.8134912252426147, 0.20236200094223022],
 8: [0.6844345927238464, 0.272461861371994],
 9: [0.6510066390037537, 0.24856504797935486]}

In [54]:
def generate_predictions(input_data):
    return generator.predict(input_data)

In [55]:
predictions = generate_predictions(np.array([[-0.000437,-60.835543]]))
predictions

1/1 [==============================] - 0s 21ms/step


array([[  3.735536,  26.2397  ,  59.107044,  40.64682 , 216.32504 ]],
      dtype=float32)

In [56]:
predictions = generate_predictions(x_train)

1128/1128 [==============================] - 2s 1ms/step


In [57]:
predictions

array([[  6.6849346,  46.313904 , 104.62996  ,  71.69409  , 381.5321   ],
       [  4.322642 ,  30.235615 ,  68.16853  ,  46.82702  , 249.2113   ],
       [  3.9528143,  27.718521 ,  62.46053  ,  42.934017 , 228.49577  ],
       ...,
       [  1.3805275,  10.207977 ,  22.776678 ,  15.841429 ,  84.23127  ],
       [  1.3673317,  10.118561 ,  22.57949  ,  15.702502 ,  83.47613  ],
       [  1.3545971,  10.032354 ,  22.39059  ,  15.568456 ,  82.74416  ]],
      dtype=float32)